The following was done on google colab some files may need modifying if working locally, the following files are not in specific order and may need to be moved, primary the cell save_modal_mask_video needs to be ran after the dataloader in 1.1

In [ ]:
#Exctract the targ.gz files, the following was done on google colab some files may need modifying if working locally
import tarfile
import os
import shutil
from glob import glob

drive_train_dir = "your file root"
temp_extract_dir = "temporary path to extract to if pulling from google drive"

drive_test_dir = "/content/drive/MyDrive/DSC_2025/LLNL_DSC/Data/test"
temp_extract_dir_test = "/content/DSC_DATA/test"

# Create a temporary writable directory
os.makedirs(temp_extract_dir, exist_ok=True)
os.makedirs(temp_extract_dir_test, exist_ok=True)


for tar_path in glob(os.path.join(drive_train_dir, "*.tar.gz")):
    print(f"Processing: {tar_path}")

    # Copy tar.gz to /content to avoid read-only issue
    local_tar_path = os.path.join("/content", os.path.basename(tar_path))
    shutil.copy(tar_path, local_tar_path)

    # Extract it to the temp writable dir
    with tarfile.open(local_tar_path, "r:gz") as tar:
        tar.extractall(path=temp_extract_dir)
    print(f"Extracted to: {temp_extract_dir}")

# for tar_path in glob(os.path.join(drive_test_dir, "*.tar.gz")):
#     print(f"Processing: {tar_path}")

#     # Copy tar.gz to /content to avoid read-only issue
#     local_tar_path = os.path.join("/content", os.path.basename(tar_path))
#     shutil.copy(tar_path, local_tar_path)

#     # Extract it to the temp writable dir
#     with tarfile.open(local_tar_path, "r:gz") as tar:
#         tar.extractall(path=temp_extract_dir_test)
#     print(f"Extracted to: {temp_extract_dir}")


In [ ]:
#Colab Cell used to extract images and convert them to videos, must be executed twice on both rgba and segmentation files
import os
import cv2
from glob import glob

def images_to_video(image_folder, output_path, fps=10):
    image_paths = sorted(glob(os.path.join(image_folder, "segmentation_*.png"))) #change rgba to segmentation or vice-vers
    if not image_paths:
        return

    # Read the first image to get frame dimensions
    first_img = cv2.imread(image_paths[0])
    if first_img is None:
        print(f"[Warning] Failed to read {image_paths[0]}")
        return
    height, width, _ = first_img.shape

    # Create video writer
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is not None:
            out.write(img)

    out.release()
    print(f"[Saved] {output_path}")

def process_all_folders(root_dir, fps=10, output_name="segmentation_output.mp4"):
    for dirpath, _, filenames in os.walk(root_dir):
        # If current folder has rgba_*.png files, make a video here
        if any(f.startswith("segmentation_") and f.endswith(".png") for f in filenames): #change rgba to segmentation
            video_path = os.path.join(dirpath, output_name)
            images_to_video(dirpath, video_path, fps=fps)

# --- Replace this with your actual path ---
root_folder = "your path"
process_all_folders(root_folder)


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm

def save_modal_mask_video(frames, output_path, fps=5):
    height, width, _ = frames[0].shape
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    for frame in frames:
        writer.write(frame)
    writer.release()

def process_all_modal_masks(root_dir, output_dir, img_size=(224, 224), fps=5):
    """
    Traverse the entire train directory and save modal mask videos (24 frames)
    for each object in each camera view of every scene.
    """
    train_dir = os.path.join(root_dir, 'train')
    all_scenes = [s for s in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, s))]

    for scene_id in tqdm(all_scenes, desc="Processing scenes"):
        scene_path = os.path.join(train_dir, scene_id)
        camera_dirs = [c for c in os.listdir(scene_path) if c.startswith('camera_')]

        for camera_id in camera_dirs:
            camera_path = os.path.join(scene_path, camera_id)
            obj_folders = [f for f in os.listdir(camera_path) if f.startswith('obj_')]

            # Get all RGBA and segmentation files
            rgba_files = sorted(glob(os.path.join(camera_path, "rgba_*.png")))

            for obj_folder in obj_folders:
                obj_path = os.path.join(camera_path, obj_folder)
                try:
                    object_id = int(obj_folder.split('_')[1])
                except:
                    continue

                frames = []
                for rgba_path in rgba_files:
                    frame_name = os.path.basename(rgba_path).replace("rgba_", "").replace(".png", "")
                    seg_path = os.path.join(camera_path, f"segmentation_{frame_name}.png")
                    if not os.path.exists(seg_path):
                        continue

                    seg_image = Image.open(seg_path)
                    seg_array = np.array(seg_image)
                    modal_mask = (seg_array == object_id).astype(np.uint8) * 255
                    modal_mask_pil = Image.fromarray(modal_mask).resize(img_size)
                    modal_mask_cv = cv2.cvtColor(np.array(modal_mask_pil), cv2.COLOR_GRAY2BGR)
                    frames.append(modal_mask_cv)


                    save_path = os.path.join(
                        output_dir,
                        scene_id,
                        camera_id,
                        obj_folder,
                        "modal_mask_video.mp4"
                    )
                    save_modal_mask_video(frames, save_path, fps=fps)

# === USAGE ===
if __name__ == "__main__":
    process_all_modal_masks(
        root_dir="",  # dataset root
        output_dir="",  # where to save videos
        img_size=(224, 224),
        fps=1
    )
